In [1]:
import os

In [2]:
import sys
import platform

print("Python version:", sys.version)
print("Platform:", platform.platform())
print("Processor:", platform.processor())


Python version: 3.12.1 (main, Mar 17 2025, 17:13:06) [GCC 9.4.0]
Platform: Linux-6.8.0-1027-azure-x86_64-with-glibc2.31
Processor: x86_64


In [3]:
!pip install psutil


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import psutil

# CPU
print("CPU cores:", psutil.cpu_count(logical=True))
print("CPU usage (%):", psutil.cpu_percent(interval=1))

# Memory
mem = psutil.virtual_memory()
print("Total memory (GB):", round(mem.total / 1e9, 2))
print("Used memory (GB):", round(mem.used / 1e9, 2))
print("Available memory (GB):", round(mem.available / 1e9, 2))

# Disk
disk = psutil.disk_usage('/')
print("Total disk (GB):", round(disk.total / 1e9, 2))
print("Used disk (GB):", round(disk.used / 1e9, 2))
print("Free disk (GB):", round(disk.free / 1e9, 2))


CPU cores: 2
CPU usage (%): 12.1
Total memory (GB): 8.32
Used memory (GB): 2.12
Available memory (GB): 5.81
Total disk (GB): 33.64
Used disk (GB): 11.45
Free disk (GB): 20.45
